# Auto Gen Tutorial - Flights Assistant
Note book written by John Adeojo
Founder, and Chief Data Scientist at [Data-centric Solutions](https://www.data-centric-solutions.com/).

---
# License

This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

## How to Credit

If you use this work or adapt it, please credit the author and the company as follows:

"Auto Gen Tutorial: Open Domain Question Answering with Wikipedia" by John Adeojo from Data-Centric Solutions, used under CC BY 4.0 / Desaturated from original

## Example Citation

In academic or professional contexts, you can cite this work as follows:

Adeojo, John. "Auto Gen Tutorial: Open Domain Question Answering with Wikipedia." Data-Centric Solutions. 13/11/2023


## 0. Utility Functions

In [1]:
import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import autogen
import openai 

# Change the directories to pick up the files. Ensure you use your own OpenAI API Keys
configurations_path = "G:/My Drive/Data-Centric Solutions/07. Blog Posts/AutoGen 2 - Flights/"

config_list = autogen.config_list_from_json(
    env_or_file="configurations.json",
    file_location=configurations_path,
    filter_dict={
        "model": ["gpt-4-1106-preview"],
        # "model": ["gpt-3.5-turbo-16k"]
    },
)
api_key = config_list[0]['api_key']
openai.api_key = api_key

## 1. Define functions

Define functions for requesting data from [Amadeus flights](https://developers.amadeus.com/self-service/category/flights/api-doc/flight-offers-search/api-reference) API.

In [3]:
import logging
from amadeus import Client, ResponseError
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('amadeus')

# Search for flight data on Amadeus flight server
def search_flight_offers_with_amadeus_client(api_key, api_secret, originLocationCode, destinationLocationCode, departureDate, 
                                             returnDate=None, adults=1, children=None, infants=None, travelClass=None, 
                                             includedAirlineCodes=None, excludedAirlineCodes=None, nonStop="false", 
                                             currencyCode=None, maxPrice=None, max=5):
    """
    Search for flight offers using Amadeus Python SDK.
    """

    # Initialize Amadeus client
    amadeus = Client(
        client_id=api_key,
        client_secret=api_secret
    )

    # Set up the parameters for the request
    params = {
        'originLocationCode': originLocationCode,
        'destinationLocationCode': destinationLocationCode,
        'departureDate': departureDate,
        'adults': adults,
        'returnDate': returnDate,
        'children': children,
        'infants': infants,
        'travelClass': travelClass,
        'includedAirlineCodes': includedAirlineCodes,
        'excludedAirlineCodes': excludedAirlineCodes,
        'nonStop': nonStop,
        'currencyCode': currencyCode,
        'maxPrice': maxPrice,
        'max': max
    }

    # Remove None values
    params = {k: v for k, v in params.items() if v is not None}
    logger.debug("Request parameters: %s", params)

    # Make the request using Amadeus client
    try:
        response = amadeus.shopping.flight_offers_search.get(**params)
        logger.debug("Response data: %s", response.data)
        return response.data
    except ResponseError as error:
        logger.error(f"An error occurred: {error}")
        logger.error(f"Error code: {error.code}")
        logger.error(f"Error message: {error.description}")
        if hasattr(error, 'response'):
            logger.error("Full error response: %s", error.response.body)  # Log full error response
        return None

def get_todays_date():
    """Returns today's date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")

# # Example usage
# today_date = get_todays_date()
# print(today_date)

# # Example usage
# api_key = api_key
# api_secret = api_secret
# offers = search_flight_offers_with_amadeus_client(api_key, api_secret, 'SYD', 'BKK', '2023-12-20', adults=1)

In [4]:
from sqlalchemy import text
from update_tabs import make_engine, apis_configs

# Execute SQL queries
def run_sql(sql_query: str):
    _, _, host, password = apis_configs()
    database = "flights_data"
    username = "john-adeojo"
    engine = make_engine(username, password, host, database)

    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            return f"query results {result.fetchall()}"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
from update_tabs import process_amadeus_data, make_engine, apis_configs

# Function to pull offers and uodate NEON PostgreSQL database
def get_flight_data(originLocationCode, destinationLocationCode, departureDate, 
                                             returnDate=None, adults=1, children=None, infants=None, travelClass=None, 
                                             includedAirlineCodes=None, excludedAirlineCodes=None, nonStop="false", 
                                             currencyCode=None, maxPrice=None, max=5):
    
    api_key, api_secret, host, password = apis_configs()
    database = "flights_data"
    username = "john-adeojo"
    
    offers = search_flight_offers_with_amadeus_client(api_key, api_secret, originLocationCode, destinationLocationCode, departureDate, 
                                             returnDate, adults, children, infants, travelClass, 
                                             includedAirlineCodes, excludedAirlineCodes, nonStop, 
                                             currencyCode, maxPrice, max=5)
    print(f"""Requesting flight data from amadues server: originLocationCode: {originLocationCode}, 
          destinationLocationCode:{destinationLocationCode}, departureDate:{departureDate},
          returnDate:{returnDate}, adults:{adults}, children:{children}, infants: {infants}, 
          travelClass: {travelClass}, includedAirlineCodes: {includedAirlineCodes}, excludedAirlineCodes:{excludedAirlineCodes},
          nonStop: {nonStop}, currencyCode: {currencyCode}, maxPrice: {maxPrice}
           """)
    
    # Process and update NEON server
    engine = make_engine(username, password, host, database)
    process_amadeus_data(offers, engine)

    # Return airport codes.
    sql_query= '''
        SELECT DISTINCT departureiatacode, arrivaliatacode
        FROM segment;
    '''
    airport_codes = run_sql(sql_query)

    return f'''Finished loading flight data to remote server /n
    The following airport IATA codes will be inportant for your SQL query: 
    {airport_codes}'''

# get_flight_data(
#                 originLocationCode='SYD', 
#                 destinationLocationCode='BKK', 
#                 departureDate='2023-12-20', 
#                 returnDate='2024-01-03', 
#                 adults=1
#                 )

## 2. Function Calls
Define the function calls

In [6]:
all_functions=[
    {
        "name": "get_flight_data",
        "description": "Pulls the flight data from the amadeus server and saves it to the neon database.",
        "parameters": {
            "type": "object",
            "properties": {
                "originLocationCode": {
                    "type": "string",
                    "description": "City/airport IATA code from which the traveler will depart, e.g., BOS for Boston"
                },
                "destinationLocationCode": {
                    "type": "string",
                    "description": "City/airport IATA code to which the traveler is going, e.g., PAR for Paris"
                },
                "departureDate": {
                    "type": "string",
                    "description": "The date on which the traveler will depart from the origin to go to the destination, in YYYY-MM-DD format"
                },
                "returnDate": {
                    "type": "string",
                    "description": "The date on which the traveler will return from the destination to the origin, in YYYY-MM-DD format"
                },
                "adults": {
                    "type": "integer",
                    "description": "The number of adult travelers (age 12 or older on the date of departure)"
                },
                "children": {
                    "type": "integer",
                    "description": "The number of child travelers (older than age 2 and younger than age 12 on the date of departure)"
                },
                "infants": {
                    "type": "integer",
                    "description": "The number of infant travelers (age 2 or younger on the date of departure)"
                },
                "travelClass": {
                    "type": "string",
                    "description": "Travel class (ECONOMY, PREMIUM_ECONOMY, BUSINESS, FIRST)"
                },
                "includedAirlineCodes": {
                    "type": "string",
                    "description": "IATA airline codes to include, comma-separated"
                },
                "excludedAirlineCodes": {
                    "type": "string",
                    "description": "IATA airline codes to exclude, comma-separated"
                },
                "nonStop": {
                    "type": "string",
                    "description": "If set to 'true', only non-stop flights are considered"
                },
                "currencyCode": {
                    "type": "string",
                    "description": "Preferred currency for the flight offers, in ISO 4217 format"
                },
                "maxPrice": {
                    "type": "integer",
                    "description": "Maximum price per traveler"
                },
                "max": {
                    "type": "integer",
                    "description": "Maximum number of flight offers to return"
                }
            },
            "required": ["originLocationCode", "destinationLocationCode", "departureDate", "adults"]
        },

        "name": "run_sql",
        "description": "Runs a SQL query against the flights_data database",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "The sql query required to pull the data needed to answer the initial query"
                }
            },
            "required": ["sql_query"]
        },
    }
]

get_flight_data_functions=[
    {
        "name": "get_flight_data",
        "description": "Pulls the flight data from the amadeus server and saves it to the neon database.",
        "parameters": {
            "type": "object",
            "properties": {
                "originLocationCode": {
                    "type": "string",
                    "description": "City/airport IATA code from which the traveler will depart, e.g., BOS for Boston"
                },
                "destinationLocationCode": {
                    "type": "string",
                    "description": "City/airport IATA code to which the traveler is going, e.g., PAR for Paris"
                },
                "departureDate": {
                    "type": "string",
                    "description": "The date on which the traveler will depart from the origin to go to the destination, in YYYY-MM-DD format"
                },
                "returnDate": {
                    "type": "string",
                    "description": "The date on which the traveler will return from the destination to the origin, in YYYY-MM-DD format"
                },
                "adults": {
                    "type": "integer",
                    "description": "The number of adult travelers (age 12 or older on the date of departure)"
                },
                "children": {
                    "type": "integer",
                    "description": "The number of child travelers (older than age 2 and younger than age 12 on the date of departure)"
                },
                "infants": {
                    "type": "integer",
                    "description": "The number of infant travelers (age 2 or younger on the date of departure)"
                },
                "travelClass": {
                    "type": "string",
                    "description": "Travel class (ECONOMY, PREMIUM_ECONOMY, BUSINESS, FIRST)"
                },
                "includedAirlineCodes": {
                    "type": "string",
                    "description": "IATA airline codes to include, comma-separated"
                },
                "excludedAirlineCodes": {
                    "type": "string",
                    "description": "IATA airline codes to exclude, comma-separated"
                },
                "nonStop": {
                    "type": "string",
                    "description": "If set to 'true', only non-stop flights are considered"
                },
                "currencyCode": {
                    "type": "string",
                    "description": "Preferred currency for the flight offers, in ISO 4217 format"
                },
                "maxPrice": {
                    "type": "integer",
                    "description": "Maximum price per traveler"
                },
                "max": {
                    "type": "integer",
                    "description": "Maximum number of flight offers to return"
                }
            },
            "required": ["originLocationCode", "destinationLocationCode", "departureDate", "adults"]
        }
    }
]

run_sql_functions=[
    {
        "name": "run_sql",
        "description": "Runs a SQL query against the flights_data database",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "The sql query required to pull the data needed to answer the initial query"
                }
            },
            "required": ["sql_query"]
        },
    }
]

In [7]:
llm_config_user_proxy = {
    "functions":all_functions,
    "config_list": config_list,
    # "request_timeout": 120,
    "seed": 100,
    "temperature": 0.7
}

llm_config_data_retriever = {
    "functions":get_flight_data_functions,
    "config_list": config_list,
    # "request_timeout": 120,
    "seed": 100,
    "temperature": 0.7
}

llm_config_analyst = {
    "functions":run_sql_functions,
    "config_list": config_list,
    # "request_timeout": 120,
    "seed": 100,
    "temperature": 0.7
}

# The llm_config_no_tools remains the same, excluding the 'functions' key.
llm_config_no_tools = {k: v for k, v in llm_config_user_proxy.items() if k != 'functions'}

In [8]:
# # Import the function from the update_tables module
# from update_tabs import process_amadeus_data, make_engine, apis_configs

# _, _, host, password = apis_configs()
# database = "flights_data"
# username = "john-adeojo"
# port = "5432"

# # Use the function
# engine = make_engine(username, password, host, database)
# json_data = offers  # Your JSON data from the Amadeus API
# process_amadeus_data(json_data, engine)

## 3. Define agent workflows

In [9]:
from messages import system_message_analyst, system_message_data_retriever, system_message_travel_agent, system_message_chat_manager, system_message_user_proxy, system_message_senior_analyst

In [12]:
import autogen 
logging.basicConfig(level=logging.WARNING)
# Configure logging level for specific loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    llm_config=llm_config_user_proxy,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=10,
    code_execution_config=False,
    system_message=system_message_user_proxy,

)

data_retriever = autogen.AssistantAgent(
    name="data_retriever",
    system_message=system_message_data_retriever,
    llm_config=llm_config_data_retriever
)

analyst = autogen.AssistantAgent(
    name="analyst",
    system_message=system_message_analyst,
    llm_config=llm_config_analyst,
    
)

travel_agent = autogen.AssistantAgent(
    name="travel_agent",
    system_message=system_message_travel_agent,
    llm_config=llm_config_no_tools,
    
)

senior_analyst = autogen.AssistantAgent(
    name="senior_analyst",
    system_message=system_message_senior_analyst,
    llm_config=llm_config_no_tools,
    
)

user_proxy.register_function(
    function_map={
        "get_flight_data": get_flight_data,
        "run_sql":run_sql,
    }
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, data_retriever, analyst, travel_agent, senior_analyst], 
    messages=[], 
    max_round=20
    )

manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config_no_tools,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    system_message=system_message_chat_manager
    )

manager.initiate_chat(
    manager, 
    message='''Show me the fast flights from London to Paris, leaving on the 
    16th of December 2023 and retruning after the 19th January 2024.'''
    )

chat_manager (to chat_manager):

Show me the fast flights from London to Paris, leaving on the 
    16th of December 2023 and retruning after the 19th January 2024.

--------------------------------------------------------------------------------
data_retriever (to chat_manager):

***** Suggested function Call: get_flight_data *****
Arguments: 
{
  "originLocationCode": "LON",
  "destinationLocationCode": "PAR",
  "departureDate": "2023-12-16",
  "returnDate": "2024-01-19",
  "adults": 1,
  "nonStop": "true"
}
****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_flight_data...
Request parameters: {'originLocationCode': 'LON', 'destinationLocationCode': 'PAR', 'departureDate': '2023-12-16', 'adults': 1, 'returnDate': '2024-01-19', 'nonStop': 'true', 'max': 5}
Request parameters: {'originLocationCode': 'LON', 'destinationLocationCode': 'PAR', 'departureDate': '2023-12-16', '

DEBUG:amadeus:Request parameters: {'originLocationCode': 'LON', 'destinationLocationCode': 'PAR', 'departureDate': '2023-12-16', 'adults': 1, 'returnDate': '2024-01-19', 'nonStop': 'true', 'max': 5}


Response data: [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-12-16', 'lastTicketingDateTime': '2023-12-16', 'numberOfBookableSeats': 7, 'itineraries': [{'duration': 'PT1H20M', 'segments': [{'departure': {'iataCode': 'LHR', 'terminal': '4', 'at': '2023-12-16T11:30:00'}, 'arrival': {'iataCode': 'CDG', 'terminal': '2E', 'at': '2023-12-16T13:50:00'}, 'carrierCode': 'AF', 'number': '1581', 'aircraft': {'code': '321'}, 'operating': {'carrierCode': 'AF'}, 'duration': 'PT1H20M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}, {'duration': 'PT1H20M', 'segments': [{'departure': {'iataCode': 'CDG', 'terminal': '2E', 'at': '2024-01-19T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '4', 'at': '2024-01-19T18:20:00'}, 'carrierCode': 'AF', 'number': '1180', 'aircraft': {'code': '319'}, 'operating': {'carrierCode': 'AF'}, 'duration': 'PT1H20M', 'id': '2', 'numberOfStops': 

DEBUG:amadeus:Response data: [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-12-16', 'lastTicketingDateTime': '2023-12-16', 'numberOfBookableSeats': 7, 'itineraries': [{'duration': 'PT1H20M', 'segments': [{'departure': {'iataCode': 'LHR', 'terminal': '4', 'at': '2023-12-16T11:30:00'}, 'arrival': {'iataCode': 'CDG', 'terminal': '2E', 'at': '2023-12-16T13:50:00'}, 'carrierCode': 'AF', 'number': '1581', 'aircraft': {'code': '321'}, 'operating': {'carrierCode': 'AF'}, 'duration': 'PT1H20M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}, {'duration': 'PT1H20M', 'segments': [{'departure': {'iataCode': 'CDG', 'terminal': '2E', 'at': '2024-01-19T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '4', 'at': '2024-01-19T18:20:00'}, 'carrierCode': 'AF', 'number': '1180', 'aircraft': {'code': '319'}, 'operating': {'carrierCode': 'AF'}, 'duration': 'PT1H20M', 'id': '2', 'nu

Requesting flight data from amadues server: originLocationCode: LON, 
          destinationLocationCode:PAR, departureDate:2023-12-16,
          returnDate:2024-01-19, adults:1, children:None, infants: None, 
          travelClass: None, includedAirlineCodes: None, excludedAirlineCodes:None,
          nonStop: true, currencyCode: None, maxPrice: None
           
user_proxy (to chat_manager):

***** Response from calling function "get_flight_data" *****
Finished loading flight data to remote server /n
    The following airport IATA codes will be inportant for your SQL query: 
    query results [('LHR', 'CDG'), ('FRA', 'LHR'), ('CDG', 'LHR'), ('SYD', 'LHR'), ('SIN', 'FRA'), ('SYD', 'SIN')]
************************************************************

--------------------------------------------------------------------------------
analyst (to chat_manager):

Given the airport IATA codes of interest, we can now construct a SQL query to find the fastest flights from London (LHR) to Paris (C